# Building Agentic Workflows with Strands Agents SDK

Introducing [Strands Agents](https://github.com/strands-agents/sdk-python), an Open Source AI Agents SDK! This is a simple-to-use, code-first framework that takes a model-driven approach to building and running AI agents in just a few lines of code, scaling from simple to complex use cases. 

The core of Strands is a simple agentic loop that connects the model and tools together, like two strands of DNA. It's already powering production AI agents in key AWS services like Amazon Q CLI, AWS Glue, and VPC Reachability Analyzer. It includes 20+ built-in tools with support for thousands of Model Context Provider (MCP) servers. AWS is excited that Accenture, Anthropic, Meta, and others are joining with support and contributions.

## Use Case
In this lab, we will create a simple restaurant reservation agent with access to basic booking tools.

![Agent Architecture](../images/strands_agent.png)

## Notebook Walk-through

In this notebook we will:
1. Create an agent using Strands Agents SDK
2. Understand Strands session handling
3. Explore agent metrics
4. Test the agent with realistic booking scenarios

In [ ]:
%store -r

In [ ]:
# Import necessary libraries
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import current_time
import uuid
import boto3
import json
import pandas as pd
from utils import create_dynamodb, selectAllFromDynamodb

In [ ]:
# Setup DynamoDB table for restaurant bookings
dynamodb = boto3.resource('dynamodb')
table_name = 'restaurant_bookings'
create_dynamodb(table_name)  # Create the table if it doesn't exist
table = dynamodb.Table(table_name)

In [ ]:
@tool
def get_booking_details(booking_id: str) -> dict:
    """
    Retrieve the details of a specific restaurant booking using its unique identifier.

    This function queries the DynamoDB table to fetch the complete information
    associated with a given booking ID. It's useful for retrieving the full
    details of a reservation, including date, name, hour, and number of guests.

    Args:
        booking_id (str): The unique identifier of the booking to retrieve.
                          This should be a string, typically an 8-character UUID.

    Returns:
        dict: A dictionary containing the booking details if found. The structure includes:
              - booking_id (str): The unique identifier of the booking
              - date (str): The date of the booking in YYYY-MM-DD format
              - name (str): The name associated with the reservation
              - hour (str): The time of the booking in HH:MM format
              - num_guests (int): The number of guests for the booking
              
              If no booking is found, it returns a dictionary with a 'message' key
              indicating that no booking was found.
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in accessing the DynamoDB table or processing the request.
                   The error is caught and returned in the response dictionary.

    Example:
        >>> get_booking_details("12345678")
        {'booking_id': '12345678', 'date': '2023-05-15', 'name': 'John Doe', 'hour': '19:30', 'num_guests': 4}
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        print(e)
        return {'error': str(e)}

In [ ]:
@tool
def create_booking(date: str, name: str, hour: str, num_guests: int) -> dict:
    """
    Create a new restaurant booking and store it in the DynamoDB table.

    This function generates a unique booking ID and creates a new entry in the
    DynamoDB table with the provided booking details. It's used to make new
    reservations in the restaurant booking system.

    Args:
        date (str): The date of the booking in YYYY-MM-DD format.
        name (str): The name to identify the reservation. Typically the guest's name.
        hour (str): The time of the booking in HH:MM format.
        num_guests (int): The number of guests for the booking.

    Returns:
        dict: A dictionary containing the newly created booking ID if successful.
              The structure is:
              - booking_id (str): The unique identifier for the new booking (8-character UUID)
              
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in generating the UUID, accessing the DynamoDB table,
                   or processing the request. The error is caught and returned in the response dictionary.

    Example:
        >>> create_booking("2023-05-15", "John Doe", "19:30", 4)
        {'booking_id': 'a1b2c3d4'}
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        print(e)
        return {'error': str(e)}

In [ ]:
@tool
def delete_booking(booking_id: str) -> dict:
    """
    Delete an existing restaurant booking from the DynamoDB table.

    This function removes a booking entry from the database based on the provided
    booking ID. It's used to cancel reservations in the restaurant booking system.

    Args:
        booking_id (str): The unique identifier of the booking to delete.
                          This should be a string, typically an 8-character UUID.

    Returns:
        dict: A dictionary containing a message indicating the result of the operation.
              If successful, the structure is:
              - message (str): A success message with the deleted booking ID
              
              If the deletion fails (but doesn't raise an exception), it returns a
              dictionary with a message indicating the failure.
              
              In case of an error, it returns a dictionary with an 'error' key
              containing the error message.

    Raises:
        Exception: If there's an error in accessing the DynamoDB table or processing the request.
                   The error is caught and returned in the response dictionary.

    Example:
        >>> delete_booking("a1b2c3d4")
        {'message': 'Booking with ID a1b2c3d4 deleted successfully'}
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


In [ ]:
# Define the agent's system prompt (instructions)
agent_instruction="""
## Role
You are a ABC Restaurant Booking agent. You are in charge of restaurant reservations.

## Instructions
- Handle restaurant reservations inquiries and requests from users
- Create new bookings when requested with appropriate details
- Retrieve booking information when asked
- Cancel reservations when requested
- Be professional and courteous in all interactions

## Output Requirements
- When responding to the end user, don't output your thinking steps
- Only give useful information to the end user
- Confirm all successful bookings, changes, and cancellations clearly
"""

In [ ]:
# Initialize the Amazon Bedrock model
model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",  # Using Amazon Nova Pro model
    max_tokens=3000,
    temperature=1,
    top_p=1,
    additional_request_fields={
        "inferenceConfig": {
            "topK": 1,
        },
    }
)

# Create the Strands Agent with our defined tools
agent = Agent(
    model=model,
    system_prompt=agent_instruction,
    tools=[current_time, get_booking_details, create_booking, delete_booking],
    callback_handler=None
)

In [ ]:
items = selectAllFromDynamodb(table_name)
items

## Testing the Agent

Now that we have created our restaurant booking agent with all the necessary tools, let's test it with some common booking scenarios.

### Scenario 1: Creating a New Reservation

First, let's test creating a new booking by having a user request a reservation.

In [ ]:
# Send a booking request to our agent
response = agent(
    """Hi, my name is Jane Doe.
    I want to book a table for 2 tomorrow at 5pm.
    """
)
print(response)

### Verify Data Storage

Let's verify that the booking was successfully added to our DynamoDB table:

In [ ]:
items = selectAllFromDynamodb(table_name)
items

## Understanding Strands Sessions & State Management

Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Key aspects of state management in Strands:

1. **Conversation History**: The complete sequence of messages between user and agent
2. **Tool State**: Information about tool executions and their results
3. **Request State**: Contextual information maintained within a single request

This state management allows the agent to maintain context across multiple interactions.

Let's examine the conversation history to see how our agent is tracking the interaction:

In [ ]:
print(json.dumps(agent.messages, indent=2))

## Analyzing Agent Metrics

Metrics are essential for understanding agent performance, optimizing behavior, and monitoring resource usage. The Strands Agents SDK provides comprehensive metrics tracking capabilities that give you visibility into:

- Number of API calls made
- Tokens used (input and output)
- Response times
- Tool usage statistics
- Model interactions

Let's examine the metrics from our agent's response:

In [ ]:
print(json.dumps(response.metrics.get_summary(), indent=2))

### Scenario 2: Retrieving Booking Details

Now let's test the agent's ability to retrieve booking information based on conversation context. The agent should be able to use the booking_id from the previous interaction.

In [ ]:
response = agent("Wait, can you get the details about the reservation I just made?")
print(response)

### Scenario 3: Cancelling a Reservation

Finally, let's test the agent's ability to cancel a reservation. The agent should be able to use the booking_id from the conversation context to delete the correct booking.

In [ ]:
response = agent("Hmm, I changed my mind, can you cancel my reservation?")
print(response)

### Verify Booking Deletion

Let's check the DynamoDB table again to confirm that the booking was successfully deleted:

In [ ]:
items = selectAllFromDynamodb(table_name)
items

## Conclusion

In this lab, we've successfully built and tested a restaurant booking agent using the Strands Agents SDK. Let's summarize what we've learned:

### Key Accomplishments
1. **Agent Creation**: We created a functional AI agent that understands natural language requests and can perform booking operations
2. **Tool Integration**: We implemented and integrated several custom tools for managing restaurant bookings:
   - Creating new reservations
   - Retrieving booking details
   - Cancelling reservations
3. **State Management**: We observed how Strands maintains conversation context across multiple interactions
4. **Metrics Analysis**: We examined performance metrics that help optimize agent behavior

### Next Steps

The Strands Agents SDK provides a powerful, flexible foundation for building intelligent agents that can handle complex workflows while maintaining contextual awareness. For Strands Agents SDK deep dive, please refer [Strands Agents examples](https://github.com/strands-agents/samples/tree/main)